In [1]:
cd(@__DIR__)
using OrdinaryDiffEq
using ModelingToolkit
using DataDrivenDiffEq
using LinearAlgebra, DiffEqSensitivity, Optim
using DiffEqFlux, Flux
using Plots, StatsPlots
using DataFrames, GLM, StatsModels
using CSV
using Interpolations
gr()

Plots.GRBackend()

In [10]:
cd("D:\\NUS_differential_equation\\SOLOW_eqn_final_training\\")

eqn1_multi= CSV.read(".\\multiplication\\Eqn1.csv",normalizenames = true)
eqn1_multi_loss= CSV.read(".\\multiplication\\Eqn1_final_loss.csv",normalizenames = true)

eqn2_multi= CSV.read(".\\multiplication\\Eqn2.csv",normalizenames = true)
eqn2_multi_loss= CSV.read(".\\multiplication\\Eqn2_final_loss.csv",normalizenames = true)

eqn3_multi= CSV.read(".\\multiplication\\Eqn3.csv",normalizenames = true)
eqn3_multi_loss= CSV.read(".\\multiplication\\Eqn3_final_loss.csv",normalizenames = true)

eqn1_add= CSV.read(".\\addition\\Eqn1.csv",normalizenames = true)
eqn1_add_loss= CSV.read(".\\addition\\Eqn1_final_loss.csv",normalizenames = true)

eqn2_add= CSV.read(".\\addition\\Eqn2.csv",normalizenames = true)
eqn2_add_loss= CSV.read(".\\addition\\Eqn2_final_loss.csv",normalizenames = true)

eqn3_add= CSV.read(".\\addition\\Eqn3.csv",normalizenames = true)
eqn3_add_loss= CSV.read(".\\addition\\Eqn3_final_loss.csv",normalizenames = true)
df= CSV.read(".\\multiplication\\compileddf.csv",normalizenames = true)
paras= CSV.read("para.csv")

,Albania,Argentina,Armenia,Australia,Azerbaijan,Bahrain,Bangladesh
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.25522,0.108467,1.05553,0.769929,1.13591,0.271731,1.15555
2,0.298486,0.987375,0.19658,0.89173,0.221021,0.989631,0.24223
3,0.05625,0.028125,0.075,0.0234375,0.075,0.0546875,0.075
4,0.04375,0.003125,0.05,-0.0015625,0.0625,0.0421875,0.05
5,-0.0298762,-0.00382839,-0.0438445,-0.012872,-0.112014,-0.00664315,0.0163367


In [3]:
eqn1_multi_loss["Argentina"]

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[3]:1
└ @ Core In[3]:1


1-element CSV.Column{Float64,Float64}:
 3.3600014758763605e-5

In [41]:
ann = FastChain(FastDense(1, 20, tanh),FastDense(20, 20, tanh),
                FastDense(20, 1))

function solownneqn(du,u,p,t)
    country= p
    newdf= getCleandataFrame(country)
    st = interpolate(newdf["srate"], BSpline(Quadratic(Reflect(OnCell()))));
    nt = interpolate(newdf["nt"], BSpline(Quadratic(Reflect(OnCell()))));
    et = interpolate(newdf["et"], BSpline(Quadratic(Reflect(OnCell()))));
    kt = interpolate(newdf["k"], BSpline(Quadratic(Reflect(OnCell()))));
    yt = interpolate(newdf["yt"], BSpline(Quadratic(Reflect(OnCell()))));
    sₜ = st(t) #savings rate
    nₜ = nt(t)
    yₜ = yt(t)
    kₜ = kt(t)
    nnp1= eqn1_multi[country]
    nnp2= eqn2_multi[country]
    nnp3= eqn3_multi[country]
    α,θ,δ,gᵦ,gₐ = paras[country]
    k = u[1]
    y=u[2]
    e=u[3]
    nn = ann(t,nnp1)[1]
    nn2= ann(t,nnp2)[1]
    nn3= ann(t,nnp3)[1]
    du[1] = sₜ*yₜ-(δ+nₜ+gᵦ)*k*nn # rate of change of kt
    du[2] = y*(gᵦ+α*du[1]/kₜ)*nn2 # rate of change of yt
    du[3] = e*(-gₐ+du[2]/y) *nn3

    return du
end


function solownneqn_add(du,u,p,t)
    country= p
    newdf= getCleandataFrame(country)
    st = interpolate(newdf["srate"], BSpline(Quadratic(Reflect(OnCell()))));
    nt = interpolate(newdf["nt"], BSpline(Quadratic(Reflect(OnCell()))));
    et = interpolate(newdf["et"], BSpline(Quadratic(Reflect(OnCell()))));
    kt = interpolate(newdf["k"], BSpline(Quadratic(Reflect(OnCell()))));
    yt = interpolate(newdf["yt"], BSpline(Quadratic(Reflect(OnCell()))));
    sₜ = st(t) #savings rate
    nₜ = nt(t)
    yₜ = yt(t)
    kₜ = kt(t)
    nnp1= eqn1_multi[country]
    nnp2= eqn2_multi[country]
    nnp3= eqn3_multi[country]
    α,θ,δ,gᵦ,gₐ = paras[country]
    k = u[1]
    y=u[2]
    e=u[3]
    nn = ann(t,nnp1)[1]
    nn2= ann(t,nnp2)[1]
    nn3= ann(t,nnp3)[1]
    du[1] = sₜ*yₜ-(δ+nₜ+gᵦ)*k+nn # rate of change of kt
    du[2] = y*(gᵦ+α*du[1]/kₜ)+nn2 # rate of change of yt
    du[3] = e*(-gₐ+du[2]/y) +nn3

    return du
end

function solownneqn1_control(du,u,p,t)
    country= p
    newdf= getCleandataFrame(country)
    st = interpolate(newdf["srate"], BSpline(Quadratic(Reflect(OnCell()))));
    nt = interpolate(newdf["nt"], BSpline(Quadratic(Reflect(OnCell()))));
    et = interpolate(newdf["et"], BSpline(Quadratic(Reflect(OnCell()))));
    kt = interpolate(newdf["k"], BSpline(Quadratic(Reflect(OnCell()))));
    yt = interpolate(newdf["yt"], BSpline(Quadratic(Reflect(OnCell()))));
    sₜ = st(t) #savings rate
    nₜ = nt(t)
    yₜ = yt(t)
    kₜ = kt(t)
    α,θ,δ,gᵦ,gₐ = paras[country]
    k = u[1]
    y=u[2]
    e=u[3]
    du[1] = sₜ*yₜ-(δ+nₜ+gᵦ)*k # rate of change of kt
    du[2] = y*(gᵦ+α*du[1]/kₜ) # rate of change of yt
    du[3] = e*(-gₐ+du[2]/yₜ)

    return du
end

solownneqn1_control (generic function with 1 method)

In [42]:
function getCleandataFrame(country)
    df_year=df["Year"][2:end]
    df1=df[string(country,"_yt")]  #GDP
    df2=df[string(country,"_kt")] #capital per capita
    df3=df[string(country,"_srate")][2:end] # savig rate
    df6= df[string(country,"_et")] #population
    df4= df[string(country,"_nt")][2:end] #population
    df8=df6[1:end-1] .-df6[2:end]#de
    df6=df6[2:end]
    df5=df2[1:end-1] .-df2[2:end] #dk
    df2=df2[2:end]
    df7= df1[1:end-1] .-df1[2:end] # dydt
    df1=df1[2:end]
    newdf= DataFrame(Year=df_year,yt=df1, k=df2, srate=df3,nt=df4,dkdt=df5,dydt=df7,dedt=df8,et=df6)
    newdf=newdf[completecases(newdf), :]
    return newdf
end

getCleandataFrame (generic function with 1 method)

In [43]:
loss_summary_multi=DataFrame()
for country in names(eqn3_multi)
    newdf= getCleandataFrame(country)
    kic = newdf["k"][1]
    yic = newdf["yt"][1]
    eic = newdf["et"][1]
    years_available=size(newdf)[1]
    nntspan = (1.0e0,years_available)
    nndatasize = years_available
    nntsteps = range(nntspan[1], nntspan[2], length=nndatasize)
    u0 = Float32[kic,yic,eic]
    prob_solownneqn = ODEProblem(solownneqn, u0, nntspan, country)
    prob_solownneqn_control = ODEProblem(solownneqn1_control, u0, nntspan, country)
#     nnexperiment = solve(prob_solownneqn, Tsit5(),p=country, abstol=1e-8, reltol=1e-8, saveat = nntsteps)
    nncontrol = solve(prob_solownneqn_control, Tsit5(),p=country, abstol=1e-8, reltol=1e-8, saveat = nntsteps)
    
    k_loss=eqn1_multi_loss[country]
    k_control= Flux.mse(nncontrol[1,:],newdf["dkdt"])
    
    y_loss=eqn2_multi_loss[country]
    y_control= Flux.mse(nncontrol[2,:],newdf["dydt"])
    
    e_loss= eqn3_multi_loss[country]
    e_control= Flux.mse(nncontrol[3,:],newdf["dedt"])
    
    loss_summary_multi[country]= [k_loss,k_control,y_loss,y_control,e_loss,e_control]
    
end

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = getCleandataFrame(::String) at In[42]:2
└ @ Main .\In[42]:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = getCleandataFrame(::String) at In[42]:3
└ @ Main .\In[42]:3
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = getCleandataFrame(::String) at In[42]:4
└ @ Main .\In[42]:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = getCleandataFrame(::String) at In[42]:5
└ @ Main .\In[42]:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = getCleandataFrame(::String) at In[42]:6
└ @ Main .\In[42]:6
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller 

In [44]:
loss_summary_add=DataFrame()
for country in names(eqn1_add)
    print(country)
    newdf= getCleandataFrame(country)
    kic = newdf["k"][1]
    yic = newdf["yt"][1]
    eic = newdf["et"][1]
    years_available=size(newdf)[1]
    nntspan = (1.0e0,years_available)
    nndatasize = years_available
    nntsteps = range(nntspan[1], nntspan[2], length=nndatasize)
    u0 = Float32[kic,yic,eic]
    prob_solownneqn = ODEProblem(solownneqn, u0, nntspan, country)
    prob_solownneqn_control = ODEProblem(solownneqn1_control, u0, nntspan, country)
#     nnexperiment = solve(prob_solownneqn, Tsit5(),p=country, abstol=1e-8, reltol=1e-8, saveat = nntsteps)
    nncontrol = solve(prob_solownneqn_control, Tsit5(),p=country, abstol=1e-8, reltol=1e-8, saveat = nntsteps)
    
    k_loss_add=eqn1_add_loss[country]
    k_control= Flux.mse(nncontrol[1,:],newdf["dkdt"])
    
#     y_loss_add=eqn2_add_loss[country]
    y_control= Flux.mse(nncontrol[2,:],newdf["dydt"])
    
#     e_loss= eqn3_multi_loss[country]
#     e_control= Flux.mse(nncontrol[3,:],newdf["dedt"])

#     try
    k_loss_add=eqn1_add_loss[country]
#     y_loss_add=eqn2_add_loss[country]

    loss_summary_add[country]= [k_loss_add,k_control]
#                 y_loss_add,y_control]  
#                     e_loss_add,e_control
            
#     catch e
#         k_loss_add=missing
#         y_loss_add=missing
#         e_loss_add=missing
#         end
    

    
end
loss_summary_add= loss_summary_add[completecases(loss_summary_add),:]

Albania

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:5
└ @ Core .\In[44]:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:6
└ @ Core .\In[44]:6
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:7
└ @ Core .\In[44]:7


Argentina

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:18
└ @ Core .\In[44]:18
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:19
└ @ Core .\In[44]:19
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:22
└ @ Core .\In[44]:22
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:28
└ @ Core .\In[44]:28
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[44]:31
└ @ Core .\In[44]:31


ArmeniaAustraliaAzerbaijanBahrainBangladeshBelarusBeninBotswanaBrazilBulgariaBurundiCambodiaCameroonCanadaCentral_African_RepublicChadChileChinaColombiaCosta_RicaCzech_RepublicDenmarkDominican_RepublicEcuadorEl_SalvadorFinlandFranceGabonGeorgiaGermanyGhanaGreeceGuatemalaGuineaHondurasHungaryIndiaIsraelItalyJamaicaJapanJordanKazakhstanKenyaKuwaitLatviaLithuaniaMadagascarMalawiMalaysiaMaliMauritaniaMauritiusMexicoMongoliaMoroccoNamibiaNepalNetherlandsNigerNigeriaNorwayOmanPakistanPanamaParaguayPeruPhilippinesPolandPortugalRomaniaRussian_FederationSaudi_ArabiaSenegalSierra_LeoneSingaporeSloveniaSouth_AfricaSpainSri_LankaSudanSwedenSwitzerlandThailandTogoTunisiaTurkeyUgandaUkraineUnited_KingdomUnited_StatesUruguayZimbabwe

,Albania,Argentina,Armenia,Australia,Azerbaijan,Bahrain
,Any,Any,Any,Any,Any,Any
1,[6.9842e-5],[0.000352984],[9.18336e-7],[0.000504643],[4.45149e-6],[0.00219639]
2,4.14475e-5,0.00042094,5.65085e-5,0.0104194,5.34798e-5,0.000768401


In [20]:
CSV.write(".//overall_performance//multi_loss.csv", loss_summary_multi)

".//overall_performance//multi_loss.csv"

In [38]:
using Statistics

println(mean(loss_summary_add[1,:]))

[0.001237069181663203]


In [40]:
println(mean(loss_summary_add[2,:]))

0.0018339504363858578


In [85]:
loss_summary_multi[1,:]

,Argentina,Azerbaijan,Bahrain,Bangladesh,Benin,Brazil
,Any,Any,Any,Any,Any,Any
1,[3.36e-5],[4.49094e-7],[0.000130287],[1.86701e-8],[5.71077e-8],[1.11888e-6]


In [86]:
loss_summary_multi[2,:]

,Argentina,Azerbaijan,Bahrain,Bangladesh,Benin,Brazil,Bulgaria
,Any,Any,Any,Any,Any,Any,Any
2,0.000420941,5.34797e-5,0.000768403,1.44277e-6,4.58423e-6,0.00018037,0.000102022


In [45]:
k_average=mean(loss_summary_multi[1,:])

1-element Array{Float64,1}:
 2.3094834277697442e-5

In [47]:
k_control =mean(loss_summary_multi[2,:])

0.0016488651696536971

In [48]:
y_average=mean(loss_summary_multi[3,:])

1-element Array{Float64,1}:
 7.212820176990014e-8

In [49]:
y_control=mean(loss_summary_multi[4,:])

0.0001453531187000637

In [50]:
e_average=mean(loss_summary_multi[5,:])

1-element Array{Float64,1}:
 0.5297710271257928

In [51]:
e_control=mean(loss_summary_multi[6,:])

1.6204827517681523e25